# Application of the Virga Sniffer

The Virga-Sniffer is a tool to detect virga (precipitation which completely evaporates before reaching the surface). As input source radar reflectivity and ceilometer cloud-base height observations are mandatory. Optionally but highly recommended are the additional information of radar mean Doppler velocity, lifting condensation level and surface rain detection.  

Virga sniffer documentation: https://virga-sniffer.readthedocs.io/en/latest/index.html

Virga sniffer code on github: https://github.com/remsens-lim/virga_sniffer/tree/main

Virga sniffer paper: https://amt.copernicus.org/articles/16/1683/2023/amt-16-1683-2023.html

## Install and import required packages

Supporting routines for reading and so are in the utils file.

Installation of the virga-sniffer via:  
python -m pip install git+https://github.com/remsens-lim/virga_sniffer.git#egg=virga_sniffer

In [ ]:
import xarray as xr
import numpy as np
from virga_sniffer import virga_mask as mvirga
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
#import json

import utils

#from virga_sniffer.utils import calc_lcl
#import metpy
#import datetime
#import matplotlib.colors as mcolors
#from matplotlib.colors import BoundaryNorm
import glob

#import numpy.ma as ma


## Define paths

In [ ]:
paths = {
    'categorize': '/projekt1/remsens/work/afoth/winter_school_HYT_2025/data/',
    #'classification': '/projekt1/remsens/work/afoth/winter_school_HYT_2025/data/',
    'vs_data': '/projekt1/remsens/work/afoth/winter_school_HYT_2025/vs_data/',
    'vs_ql': '/projekt1/remsens/work/afoth/winter_school_HYT_2025/vs_plots/',
    'data_collection': '/projekt1/remsens/work/afoth/winter_school_HYT_2025/data_collection/'
}

## Prepare folder structure 

Prepare the folder structure by the following python script such that the structure looks like:

```
project  # winter_school_HYT_2025
│   apply_virga-sniffer.ipynb    
│
└───data
│   │   yyyymmdd_hyytiala_categorize.nc
│   │   ...
|
└───vs_data
│   │   ...
|
└───vs_plots
│   │   ...
|
└───data_collection  # data from the others in the course
    │   ...
```

In [ ]:
for path in paths:
    if not os.path.exists(paths[path]):
        os.makedirs(paths[path])

## Interisting dates

from cloudnet quiicklook:
- 2024-02-04
- 2024-02-05
- 2024-02-06
- 2024-02-07
- 2024-02-11
- 2024-02-13
- 2024-02-14

## Define the date range to process

In [ ]:
date_start = '2024-02-04'
date_end = '2024-02-07'

date_range = pd.date_range(start=date_start,end=date_end).strftime("%Y-%m-%d")

## Download required data

Download required data. In this case, these are only the categorize files for the specified days.

In [ ]:
utils.download_cloudnet_categorize(date_start, date_end, paths)


## Meta data and configs

Here you find all infomration needed for the configuration: https://virga-sniffer.readthedocs.io/en/latest/config.html

default values: **default was optimized for subtropical Atlantic**  
```python
DEFAULT_CONFIG = dict(  
    smooth_window_cbh=60,  # [s] smoothing of CBH  
    smooth_window_lcl=300,  # [s] smoothing of LCL if provided  
    require_cbh=True,  # need a cloud base to be considered as virga?  
    mask_rain=True,  # apply rain mask from ancillary data?  
    mask_rain_ze=True,  # apply rain mask from radar signal?  
    ze_thres=0,  # [dBz] minimum Radar signal at lowest range-gate which is considered rain  
    ignore_virga_gaps=True,  # ignore gaps in virga when masking, if False, no virga after first gap (top-down)  
    minimum_rangegate_number=2,  # minimum number of range-gates in column to be considered virga  
    mask_vel=True,  # apply velocity mask ?  
    vel_thres=0,  # [ms-1] velocity threshold  
    mask_clutter=True,  # apply clutter threshold line ?  
    clutter_c=-8,  # [ms-1] intercept of clutter threshold line  
    clutter_m=4,  # [ms-1 dBz-1] slope of clutter threshold line  
    layer_threshold=500,  # [m] cbh layer separation  
    cloud_max_gap=150,  # [m] maximum gap between virga signal to count as connected virga and for clouds to cloud base  
    clean_threshold=0.05,  # [0-1] remove cbh layer if below (clean_treshold*100)% of total data  
    cbh_layer_fill=True,  # fill gaps of cbh layer?  
    cbh_fill_method='slinear',  # fill method of cbh layer gaps  
    layer_fill_limit=60,  # [s] fill gaps of cbh layer with this gap limit  
    cbh_ident_function=[1, 0, 2, 0, 3, 1, 0, 2, 0, 3, 4])  # order of operations applied to cbh: 0-clean, 1-split, 2-merge, 3-add-LCL, 4 smooth  
```

In [ ]:
#site_meta = {'name': 'Hyytiälä', 'altitude': 150}

name = 'Andreas'

config = {
    "require_cbh": True,
    "mask_vel": True,
    "mask_clutter": True,
    "mask_rain": True,
    #"mask_rain_ze": False,
    "minimum_rangegate_number": 4,
    "cloud_max_gap": 150,
    "precip_max_gap": 500,
    #"vel_thres": 2,
    #"ze_thres": 0,
    "clutter_c": -8,
    "clutter_m": 4,
    "cbh_smooth_window": 600,
    #"lcl_replace_cbh": False,
    #"lcl_smooth_window": 300,
    "cbh_layer_thres": 500,
    "cbh_connect2top": False,
    #"cbh_clean_thres": 0.025,
    "cbh_fill_method": "slinear",
    "cbh_fill_limit": 300,
    "cbh_processing": [1, 0, 2, 0, 1, 0, 2, 0, 4]
}

## Define input data and run virga sniffer

radar reflectivity and Doppler velocities from cloudnet categorize and liquid layer base determined from target classification

Set **h_bottom = 1**, if the **lowest radar range gate is empty**, as it is the case here for the Hyytiälä radar. The rain mask will not work in case of an empty lowest radar range gate.

In [ ]:
for date in date_range:
    print(date)

    #try:
    #ceilo_dat = proc_ceilo(date, paths, site_meta)
    h_bottom = 1  # lowest radar range gate with values
    categ = utils.proc_categ(date, paths).isel(height=slice(h_bottom, -1))
    #classi = proc_class(date, paths).isel(height=slice(h_bottom, -1))

    # liquid containing layer base height from cloudnet target classifiaction
    lcbh = utils.find_lowest_liquid_containing_layer_height(categ)

    # wind from ECMWF (within categorize)
    #uwind = categ.uwind.interp(model_time=categ.time, model_height=categ.height)
    #vwind = categ.vwind.interp(model_time=categ.time, model_height=categ.height)

    # surface rain flag from Parsivel2 stored in categorize data
    flag_rain_ground = categ.rain_detected.rolling(time=5, center=False).max() > 0
    #flag_rain_ground = categ.rainfall_rate > 0

    time = categ.time
    height = categ.height
    
    # Preparing the Input
    input_ds = xr.Dataset(
        {
          "Ze": (('time', 'range'), categ.Z.data),
          # cloud_base data needs two dimensions, even if its only one layer
          "cloud_base_height": (('time', 'layer'), np.array(lcbh.data)[:, np.newaxis]),
          "flag_surface_rain": (('time'), flag_rain_ground.data),
          "vel": (('time', 'range'), categ.v.data),
          #"category_bit": (('time', 'range'), categ.category_bits.data),
          #"target_classification": (('time', 'range'), classi.target_classification.data),
          #"uwind": (('time', 'range'), uwind.data),
          #"vwind": (('time', 'range'), vwind.data),
       },
       coords={
          "time": ('time', time.data),
          "range": ('range', height.data),
          "layer": ('layer', [0])
       }
    )

    # Restrict plots to 'time_step' hours
    time_step = 24
    for dtime in range(0,24,time_step):

        dsp = input_ds.sel(time = slice(np.datetime64(date)+np.timedelta64(dtime,'h'),
                                     np.datetime64(date)+np.timedelta64(dtime+time_step,'h')))
        #print(config)
        dsout = mvirga(dsp, config)
                
        year, month, day = utils.date2filestring(date)

        # save vs output file
        dsout.to_netcdf(paths['vs_data'] + '/' + year + month + day + str(dtime).zfill(2) + "_virga-sniffer-hyt.nc")
        
        # plot vs
        fig, axs, aaa = dsout.vsplot.quicklook_full(radar='JOYRAD-94')
        fig.savefig(paths['vs_ql'] + '/'  + year + month + day + str(dtime).zfill(2) + "_ql",dpi=100)
        

display virga sniffer output file

In [ ]:
dsout

# Get statistics from the virga-sniffer output

## Merge single files

Please check beforehand that you do not enter some time steps twice if, for example, there is a file with 24 hours and 8 with three hours each in your folder because you have tried different time steps.  
**Solutions:**
- delete old files
- use file naming patterns below 

In [ ]:
file_list = sorted(glob.glob(paths['vs_data'] + "202402*00_virga-sniffer-hyt.nc"))

data_int_all = []
for vs_file in file_list:
    data_int = xr.open_dataset(vs_file)
    data_int_all.append(data_int)

data_all = xr.concat(data_int_all, dim='time')

Add your name as a variable so that we can merge everyone's files later and the individual files can be accessed as a dimension.

In [ ]:
data_all['name'] = name

Write data which includes all processed days into the data_collection folder

In [ ]:
data_all.to_netcdf(paths['data_collection'] + 'virga_sniffer_data_' + str(data_all['name'].values) + '.nc')

## Virga fraction

In [ ]:
print(str(int(data_all['flag_virga_layer'].isel(layer=0).sum().item())) + ' number of time steps with detected virga in the lowest layer')
print(str(int(data_all['flag_precip_layer'].isel(layer=0).sum().item())) + ' number of time steps with detected precipitation in the lowest layer')
print(str(data_all['flag_surface_rain'].sum().item()) + ' number of time steps with surface precipitation')

In [ ]:
print('Virga fraction')
print(data_all['flag_virga_layer'].isel(layer=0).sum().item()/data_all['flag_precip_layer'].isel(layer=0).sum().item())

## Virga depth

In [ ]:
bins = np.arange(0,3000, 100)

data_all.isel(layer=0).virga_depth.plot.hist(bins=bins, histtype='step', lw=2)
data_all.isel(layer=1).virga_depth.plot.hist(bins=bins, histtype='step', lw=2)
data_all.isel(layer=2).virga_depth.plot.hist(bins=bins, histtype='step', lw=2)

# Merge Virga sniffer files from the entire course

At this point it is necessary to share your virga_sniffer_data_[name].nc file with the other. 

**I NEED TO FIND A SOLUTION FOR DATA SHARING. MAYBE A FOLDER ON OUR SPECIHERWOLKE**

In [ ]:
file_list = glob.glob(paths['data_collection'] + 'virga_sniffer_data_*.nc')

data_all_course_list = []

for file in file_list:
    data_all_course_list.append(xr.open_dataset(file))

In [ ]:
data_all_course = xr.concat(data_all_course_list, dim='name')

display data

In [ ]:
data_all_course

In [ ]:
for i, i_name in enumerate(data_all_course['name']):
    data_all_course.isel(layer=0, name=i).virga_depth.plot.hist(bins=bins, histtype='step', lw=2, label=i_name)
plt.legend(frameon=False)


In [ ]:
for i, i_name in enumerate(data_all_course['name']):
    data = {
        'Name': str(i_name.values),
        'Number of virga': [data_all_course['flag_virga_layer'].isel(layer=0, name=i).sum().item()],
        'Number of precipation': [data_all_course['flag_precip_layer'].isel(layer=0, name=i).sum().item()], 
        'Number of surface precip': [data_all_course['flag_surface_rain'].isel(name=i).sum().item()], 
        'Virga fraction': data_all_course['flag_virga_layer'].isel(layer=0, name=i).sum().item() / data_all_course['flag_precip_layer'].isel(layer=0, name=i).sum().item()
    }

df_numbers = pd.DataFrame(data)


In [ ]:
df_numbers

In [ ]:
df_numbers.describe()

## Comparison of configurations

- prepare and save the config file in the data_collection folder
- load all json from other investigators
- merge config dataframes 

In [ ]:
dict_config = utils.prepare_and_save_config(name, config, paths)

df_config_list = utils.load_all_json_configs(paths)

df_configs_all = pd.concat(df_config_list).set_index('name').T

Table containing the configurations of all investigators

In [ ]:
df_configs_all
